In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import linprog


In [82]:
df = pd.read_csv("Cleaned_Indian_Food_Nutrition_Per_100kcal.csv")
df

,Dish Name,Calories (kcal),Carbohydrates (g) per 100kcal,Protein (g) per 100kcal,Fats (g) per 100kcal,Free Sugar (g) per 100kcal,Fibre (g) per 100kcal,Sodium (mg) per 100kcal,Calcium (mg) per 100kcal,Iron (mg) per 100kcal,Vitamin C (mg) per 100kcal
0,Hot tea (Garam Chai),16.14,15.985130,2.416357,3.283767,15.985130,0.000000,19.330855,87.980173,0.123916,3.097893
1,Instant coffee,23.16,15.759931,2.763385,3.238342,15.630397,0.000000,21.243523,90.112263,0.259067,6.519862
2,Espreso coffee,51.54,12.844393,3.395421,4.152115,12.669771,0.000000,27.124563,112.727978,0.291036,2.929763
3,Iced tea,10.34,26.112186,0.290135,0.096712,26.112186,0.000000,2.224371,11.411992,0.193424,57.543520
4,Raw mango drink (Aam panna),35.92,25.194878,0.445434,0.083519,20.851893,1.698218,222.216036,19.710468,0.389755,126.113586
...,...,...,...,...,...,...,...,...,...,...,...
1009,Sweet and sour tomato pickle (Khatta meetha ta...,60.88,10.758870,2.069645,5.321945,7.079501,3.613666,2105.683311,24.934297,0.886991,NaN
1010,Jhatpat achar with carrot (Jhatpat achaar gaja...,91.21,6.929065,2.170815,7.181230,3.332968,5.569565,2266.560684,59.543910,2.543581,NaN
1011,Tomato chutney (Tamatar ki chutney),176.07,18.089396,0.550917,3.413415,17.050037,0.846254,467.796899,14.392003,0.545238,NaN
1012,Tomato ketchup,33.07,19.594799,2.751739,0.907167,14.151799,5.745389,538.161476,46.356214,1.088600,NaN


In [83]:
df.columns

Index(['Dish Name', 'Calories (kcal)', 'Carbohydrates (g) per 100kcal',
       'Protein (g) per 100kcal', 'Fats (g) per 100kcal',
       'Free Sugar (g) per 100kcal', 'Fibre (g) per 100kcal',
       'Sodium (mg) per 100kcal', 'Calcium (mg) per 100kcal',
       'Iron (mg) per 100kcal', 'Vitamin C (mg) per 100kcal'],
      dtype='object')

In [86]:
food_names = df["Dish Name"].dropna().unique().tolist()

In [ ]:
food_names

['Hot tea (Garam Chai)',
 'Instant coffee',
 'Espreso coffee',
 'Iced tea',
 'Raw mango drink (Aam panna)',
 'Fruit Punch (with fresh juices)',
 'Fruit Punch (with squashes)',
 'Lemonade',
 'Lem-o-gin',
 'Cumin infused water (Jeere/Zeere ka pani)',
 'Coco pine cooler',
 'Summer cooler',
 'Hot cocoa',
 'Cold coffee with ice cream',
 'Banana milkshake (Kele milkshake)',
 'Mango milkshake (Aam milkshake)',
 'Pineapple milkshake (Ananas milkshake)',
 'Orange milkshake (Narangi milkshake)',
 'Egg nog',
 'Sweet Lassi (Meethi lassi)',
 'Lassi (salted)',
 'Cheese and chilli sandwich',
 'Egg sandwich (Ande ka sandwich)',
 'Cucumber sandwich (Kheere ka sandwich)',
 'Cheese and pineapple sandwich (Cheese aur ananas ka sandwich)',
 'Cheese and tomato sandwich (Cheese aur tamatar ke sandwich)',
 'Chicken sandwich',
 'Peanut and tomato sandwich (Moongfali aur tamatar ka sandwich)',
 'Rainbow sandwich',
 'Club sandwich',
 'Vegetarian club sandwich',
 'Pin wheel sandwich',
 'Carrot apple sandwich (Gaj

In [103]:
from openai import OpenAI
import pandas as pd
import json
import time
from tqdm import tqdm
from dotenv import load_dotenv
import os

In [104]:
# 🔐 Load your .env file
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

In [105]:
# 📥 Load dataset
df = pd.read_csv("Cleaned_Indian_Food_Nutrition_Per_100kcal.csv")
food_names = df["Dish Name"].dropna().unique().tolist()

In [106]:
# 🔁 Split list into chunks
def chunk_list(lst, size):
    for i in range(0, len(lst), size):
        yield lst[i:i + size]

In [107]:
batches = list(chunk_list(food_names, 30))

In [108]:
# 🧠 Prompt builder
def build_prompt(batch):
    food_list = "\n".join([f"{i+1}. {item}" for i, item in enumerate(batch)])
    return (
        "Classify the following Indian food items. For each item, return:\n"
        "- Whether it's Veg or Non-Veg\n"
        "- Suitable meal time: Breakfast, Lunch, Dinner, or Any\n"
        "Return the result as a JSON list of dictionaries with keys: Food Name, Type, Recommended Meal.\n\n"
        f"Food List:\n{food_list}"
    )

In [109]:
results = []

In [110]:
# 🧠 Call LLM on each batch
for batch in tqdm(batches, desc="🔍 Classifying Food Items"):
    prompt = build_prompt(batch)
    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.2,
            max_tokens=1000
        )
        content = response.choices[0].message.content
        parsed = json.loads(content)
        results.extend(parsed)
        time.sleep(1.2)
    except Exception as e:
        print(f"❌ Error in batch {batch[:3]}...: {e}")
        continue

🔍 Classifying Food Items:  18%|█▊        | 6/34 [03:01<14:51, 31.82s/it]

❌ Error in batch ['Kidney bean curry (Rajmah curry)', 'Sambar', 'Besan kadhi with pakodies']...: Unterminated string starting at: line 31 column 51 (char 3066)


🔍 Classifying Food Items:  53%|█████▎    | 18/34 [08:00<06:17, 23.61s/it]

❌ Error in batch ['Spaghetti with meat balls and tomato sauce', 'Spaghetti with paneer balls and tomato sauce', 'Lasagne with meat sauce']...: Expecting ',' delimiter: line 32 column 1 (char 3145)


🔍 Classifying Food Items: 100%|██████████| 34/34 [15:08<00:00, 26.73s/it]


In [111]:
results

[{'Food Name': 'Hot tea (Garam Chai)',
  'Type': 'Veg',
  'Recommended Meal': 'Any'},
 {'Food Name': 'Instant coffee', 'Type': 'Veg', 'Recommended Meal': 'Any'},
 {'Food Name': 'Espresso coffee', 'Type': 'Veg', 'Recommended Meal': 'Any'},
 {'Food Name': 'Iced tea', 'Type': 'Veg', 'Recommended Meal': 'Any'},
 {'Food Name': 'Raw mango drink (Aam panna)',
  'Type': 'Veg',
  'Recommended Meal': 'Any'},
 {'Food Name': 'Fruit Punch (with fresh juices)',
  'Type': 'Veg',
  'Recommended Meal': 'Any'},
 {'Food Name': 'Fruit Punch (with squashes)',
  'Type': 'Veg',
  'Recommended Meal': 'Any'},
 {'Food Name': 'Lemonade', 'Type': 'Veg', 'Recommended Meal': 'Any'},
 {'Food Name': 'Lem-o-gin', 'Type': 'Veg', 'Recommended Meal': 'Any'},
 {'Food Name': 'Cumin infused water (Jeere/Zeere ka pani)',
  'Type': 'Veg',
  'Recommended Meal': 'Any'},
 {'Food Name': 'Coco pine cooler', 'Type': 'Veg', 'Recommended Meal': 'Any'},
 {'Food Name': 'Summer cooler', 'Type': 'Veg', 'Recommended Meal': 'Any'},
 {'Food

In [112]:
# 🧾 Convert to DataFrame
tagged_df = pd.DataFrame(results)
tagged_df.columns = [col.strip() for col in tagged_df.columns]

In [113]:
# 🔁 Merge with original dataset
final_df = df.merge(tagged_df, how="left", left_on="Dish Name", right_on="Food Name")


In [117]:
final_df.head()

,Dish Name,Calories (kcal),Carbohydrates (g) per 100kcal,Protein (g) per 100kcal,Fats (g) per 100kcal,Free Sugar (g) per 100kcal,Fibre (g) per 100kcal,Sodium (mg) per 100kcal,Calcium (mg) per 100kcal,Iron (mg) per 100kcal,Vitamin C (mg) per 100kcal,Food Name,Type,Recommended Meal
0,Hot tea (Garam Chai),16.14,15.985130,2.416357,3.283767,15.985130,0.000000,19.330855,87.980173,0.123916,3.097893,Hot tea (Garam Chai),Veg,Any
1,Instant coffee,23.16,15.759931,2.763385,3.238342,15.630397,0.000000,21.243523,90.112263,0.259067,6.519862,Instant coffee,Veg,Any
2,Espreso coffee,51.54,12.844393,3.395421,4.152115,12.669771,0.000000,27.124563,112.727978,0.291036,2.929763,NaN,NaN,NaN
3,Iced tea,10.34,26.112186,0.290135,0.096712,26.112186,0.000000,2.224371,11.411992,0.193424,57.543520,Iced tea,Veg,Any
4,Raw mango drink (Aam panna),35.92,25.194878,0.445434,0.083519,20.851893,1.698218,222.216036,19.710468,0.389755,126.113586,Raw mango drink (Aam panna),Veg,Any


In [192]:
# 💾 Save result
final_df.to_csv("Tagged_Indian_Food_Nutrition.csv", index=False)
print("✅ Saved as Tagged_Indian_Food_Nutrition.csv")

✅ Saved as Tagged_Indian_Food_Nutrition.csv


In [193]:
df = pd.read_csv("Tagged_Indian_Food_Nutrition.csv")

In [194]:
df.head()

,Dish Name,Calories (kcal),Carbohydrates (g) per 100kcal,Protein (g) per 100kcal,Fats (g) per 100kcal,Free Sugar (g) per 100kcal,Fibre (g) per 100kcal,Sodium (mg) per 100kcal,Calcium (mg) per 100kcal,Iron (mg) per 100kcal,Vitamin C (mg) per 100kcal,Food Name,Type,Recommended Meal
0,Hot tea (Garam Chai),16.14,15.985130,2.416357,3.283767,15.985130,0.000000,19.330855,87.980173,0.123916,3.097893,Hot tea (Garam Chai),Veg,Any
1,Instant coffee,23.16,15.759931,2.763385,3.238342,15.630397,0.000000,21.243523,90.112263,0.259067,6.519862,Instant coffee,Veg,Any
2,Espreso coffee,51.54,12.844393,3.395421,4.152115,12.669771,0.000000,27.124563,112.727978,0.291036,2.929763,NaN,NaN,NaN
3,Iced tea,10.34,26.112186,0.290135,0.096712,26.112186,0.000000,2.224371,11.411992,0.193424,57.543520,Iced tea,Veg,Any
4,Raw mango drink (Aam panna),35.92,25.194878,0.445434,0.083519,20.851893,1.698218,222.216036,19.710468,0.389755,126.113586,Raw mango drink (Aam panna),Veg,Any


In [195]:
# ✅ Clean column names for uniformity
df.columns = [col.strip().replace(" ", "_").replace("(", "").replace(")", "").lower() for col in df.columns]

In [196]:
# ✅ Rename key LLM-generated columns
df.rename(columns={
    "food_name": "food_name_llm",
    "type": "veg_nonveg",
    "recommended_meal": "meal_time"
}, inplace=True)

In [197]:
# ✅ Drop rows with missing core nutrient info
df = df.dropna(subset=[
    "protein_g_per_100kcal", "carbohydrates_g_per_100kcal", "fats_g_per_100kcal"
])

In [198]:

# ✅ Reset index
df = df.reset_index(drop=True)

# ✅ Preview cleaned data
df.head()

,dish_name,calories_kcal,carbohydrates_g_per_100kcal,protein_g_per_100kcal,fats_g_per_100kcal,free_sugar_g_per_100kcal,fibre_g_per_100kcal,sodium_mg_per_100kcal,calcium_mg_per_100kcal,iron_mg_per_100kcal,vitamin_c_mg_per_100kcal,food_name_llm,veg_nonveg,meal_time
0,Hot tea (Garam Chai),16.14,15.985130,2.416357,3.283767,15.985130,0.000000,19.330855,87.980173,0.123916,3.097893,Hot tea (Garam Chai),Veg,Any
1,Instant coffee,23.16,15.759931,2.763385,3.238342,15.630397,0.000000,21.243523,90.112263,0.259067,6.519862,Instant coffee,Veg,Any
2,Espreso coffee,51.54,12.844393,3.395421,4.152115,12.669771,0.000000,27.124563,112.727978,0.291036,2.929763,NaN,NaN,NaN
3,Iced tea,10.34,26.112186,0.290135,0.096712,26.112186,0.000000,2.224371,11.411992,0.193424,57.543520,Iced tea,Veg,Any
4,Raw mango drink (Aam panna),35.92,25.194878,0.445434,0.083519,20.851893,1.698218,222.216036,19.710468,0.389755,126.113586,Raw mango drink (Aam panna),Veg,Any


In [199]:
# ✅ Normalize macro and micro nutrients per 1 kcal
df["protein_per_kcal"]     = df["protein_g_per_100kcal"] / 100
df["carbs_per_kcal"]       = df["carbohydrates_g_per_100kcal"] / 100
df["fat_per_kcal"]         = df["fats_g_per_100kcal"] / 100
df["sugar_per_kcal"]       = df["free_sugar_g_per_100kcal"] / 100
df["fiber_per_kcal"]       = df["fibre_g_per_100kcal"] / 100
df["sodium_per_kcal"]      = df["sodium_mg_per_100kcal"] / 100
df["calcium_per_kcal"]     = df["calcium_mg_per_100kcal"] / 100
df["iron_per_kcal"]        = df["iron_mg_per_100kcal"] / 100
df["vitamin_c_per_kcal"]   = df["vitamin_c_mg_per_100kcal"] / 100

# ✅ Preview
df[[
    "dish_name", "protein_per_kcal", "carbs_per_kcal", "fat_per_kcal",
    "fiber_per_kcal", "sugar_per_kcal", "sodium_per_kcal"
]].head()


,dish_name,protein_per_kcal,carbs_per_kcal,fat_per_kcal,fiber_per_kcal,sugar_per_kcal,sodium_per_kcal
0,Hot tea (Garam Chai),0.024164,0.159851,0.032838,0.000000,0.159851,0.193309
1,Instant coffee,0.027634,0.157599,0.032383,0.000000,0.156304,0.212435
2,Espreso coffee,0.033954,0.128444,0.041521,0.000000,0.126698,0.271246
3,Iced tea,0.002901,0.261122,0.000967,0.000000,0.261122,0.022244
4,Raw mango drink (Aam panna),0.004454,0.251949,0.000835,0.016982,0.208519,2.222160


In [200]:
df = df[
    (df["protein_g_per_100kcal"] <= 25) &
    (df["fats_g_per_100kcal"] <= 20) &
    (df["carbohydrates_g_per_100kcal"] <= 40)
].copy()


In [201]:
# 🧂 Condiment Filtering: restrict portion sizes for sauces, chutneys, etc.
restrict_keywords = [
    "sauce", "chutney", "pickle", "jam", "ketchup", "achar", "achaar",
    "murabba", "marmalade", "preserve", "squash", "puree", "relish", "dip"
]

In [202]:
df.head()

,dish_name,calories_kcal,carbohydrates_g_per_100kcal,protein_g_per_100kcal,fats_g_per_100kcal,free_sugar_g_per_100kcal,fibre_g_per_100kcal,sodium_mg_per_100kcal,calcium_mg_per_100kcal,iron_mg_per_100kcal,...,meal_time,protein_per_kcal,carbs_per_kcal,fat_per_kcal,sugar_per_kcal,fiber_per_kcal,sodium_per_kcal,calcium_per_kcal,iron_per_kcal,vitamin_c_per_kcal
0,Hot tea (Garam Chai),16.14,15.985130,2.416357,3.283767,15.985130,0.000000,19.330855,87.980173,0.123916,...,Any,0.024164,0.159851,0.032838,0.159851,0.000000,0.193309,0.879802,0.001239,0.030979
1,Instant coffee,23.16,15.759931,2.763385,3.238342,15.630397,0.000000,21.243523,90.112263,0.259067,...,Any,0.027634,0.157599,0.032383,0.156304,0.000000,0.212435,0.901123,0.002591,0.065199
2,Espreso coffee,51.54,12.844393,3.395421,4.152115,12.669771,0.000000,27.124563,112.727978,0.291036,...,NaN,0.033954,0.128444,0.041521,0.126698,0.000000,0.271246,1.127280,0.002910,0.029298
3,Iced tea,10.34,26.112186,0.290135,0.096712,26.112186,0.000000,2.224371,11.411992,0.193424,...,Any,0.002901,0.261122,0.000967,0.261122,0.000000,0.022244,0.114120,0.001934,0.575435
4,Raw mango drink (Aam panna),35.92,25.194878,0.445434,0.083519,20.851893,1.698218,222.216036,19.710468,0.389755,...,Any,0.004454,0.251949,0.000835,0.208519,0.016982,2.222160,0.197105,0.003898,1.261136


In [203]:
df.columns

Index(['dish_name', 'calories_kcal', 'carbohydrates_g_per_100kcal',
       'protein_g_per_100kcal', 'fats_g_per_100kcal',
       'free_sugar_g_per_100kcal', 'fibre_g_per_100kcal',
       'sodium_mg_per_100kcal', 'calcium_mg_per_100kcal',
       'iron_mg_per_100kcal', 'vitamin_c_mg_per_100kcal', 'food_name_llm',
       'veg_nonveg', 'meal_time', 'protein_per_kcal', 'carbs_per_kcal',
       'fat_per_kcal', 'sugar_per_kcal', 'fiber_per_kcal', 'sodium_per_kcal',
       'calcium_per_kcal', 'iron_per_kcal', 'vitamin_c_per_kcal'],
      dtype='object')

In [204]:
# 🧮 Add realistic portion bounds for each food
df["portion_bounds"] = df["dish_name"].apply(
    lambda name: (0, 1) if any(k in name.lower() for k in restrict_keywords) else (0, 4)
)

In [205]:
# 🧁 Fill missing optional health values with 0 using actual column names
for col in ["free_sugar_g_per_100kcal", "fibre_g_per_100kcal", "sodium_mg_per_100kcal"]:
    df[col] = df[col].fillna(0)


In [206]:
from openai import OpenAI
import os
import json

client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

def extract_nutrition_preferences(natural_input: str):
    system_msg = """
You are a dietitian AI that extracts structured nutrition goals from natural language.

Return a JSON object with:
- calories_target: integer
- protein_pct, carbs_pct, fat_pct: percentages adding up to 100
- fiber_min: minimum daily grams of fiber (default 25)
- sugar_max: max grams of sugar (default 25)
- sodium_max: max mg of sodium (default 2300)

Only include numbers. If not specified, use defaults.
"""
    user_msg = f"Input: {natural_input}"

    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": system_msg},
            {"role": "user", "content": user_msg}
        ],
        temperature=0
    )

    extracted_json = response.choices[0].message.content.strip()
    return json.loads(extracted_json)


In [207]:
extract_nutrition_preferences("Give me a 1800 calorie meal with 40% carbs, 30% protein, 30% fat. Low sugar and high fiber.")


{'calories_target': 1800,
 'protein_pct': 30,
 'carbs_pct': 40,
 'fat_pct': 30,
 'fiber_min': 25,
 'sugar_max': 25,
 'sodium_max': 2300}

In [208]:
user_text_input="Give me a 1800 calorie meal with 40% carbs, 30% protein, 30% fat. Low sugar and high fiber."

In [209]:
params = extract_nutrition_preferences(user_text_input)

calories_target = params["calories_target"]
protein_pct     = params["protein_pct"]
carbs_pct       = params["carbs_pct"]
fat_pct         = params["fat_pct"]
fiber_min       = params.get("fiber_min", 25)
sugar_max       = params.get("sugar_max", 25)
sodium_max      = params.get("sodium_max", 2300)


In [210]:
# 🧠 Convert percentages to grams (4 kcal/g for protein & carbs, 9 kcal/g for fat)
protein_target = (calories_target * (protein_pct / 100)) / 4
carbs_target = (calories_target * (carbs_pct / 100)) / 4
fat_target = (calories_target * (fat_pct / 100)) / 9

In [211]:
# # 🎯 Take user input
# calories_target = float(input("Enter your daily calorie goal (e.g., 1800): "))
# protein_pct = float(input("Percent of calories from protein (e.g., 30): "))
# carbs_pct = float(input("Percent of calories from carbs (e.g., 40): "))
# fat_pct = float(input("Percent of calories from fat (e.g., 30): "))

# # 🧠 Convert percentages to grams (4 kcal/g for protein & carbs, 9 kcal/g for fat)
# protein_target = (calories_target * (protein_pct / 100)) / 4
# carbs_target = (calories_target * (carbs_pct / 100)) / 4
# fat_target = (calories_target * (fat_pct / 100)) / 9

# # Optional extra constraints
# fiber_min = 25
# sugar_max = 25
# sodium_max = 2200


In [212]:
print("calories ", calories_target)                
print("Protein ≥", protein_target)
print("Carbs ≥", carbs_target)
print("Fat ≥", fat_target)
print("Fiber ≥", fiber_min)
print("Sugar ≤", sugar_max)
print("Sodium ≤", sodium_max)


calories  1800
Protein ≥ 135.0
Carbs ≥ 180.0
Fat ≥ 60.0
Fiber ≥ 25
Sugar ≤ 25
Sodium ≤ 2300


In [213]:
# ✅ Show user their daily macro goals
print("\n🎯 Daily Macronutrient Targets:")
print(f"Calories: {calories_target} kcal")
print(f"Protein:  {protein_target} g")
print(f"Carbs:    {carbs_target} g")
print(f"Fat:      {fat_target} g")


🎯 Daily Macronutrient Targets:
Calories: 1800 kcal
Protein:  135.0 g
Carbs:    180.0 g
Fat:      60.0 g


In [214]:
# ⚠️ Validate
if round(protein_pct + carbs_pct + fat_pct, 1) != 100:
    raise ValueError("❌ The macronutrient percentages must sum to 100%.")

In [215]:
from scipy.optimize import linprog
import numpy as np

# === Extract nutrients using your actual column names ===
protein = df["protein_g_per_100kcal"].values
carbs   = df["carbohydrates_g_per_100kcal"].values
fat     = df["fats_g_per_100kcal"].values
fiber   = df["fibre_g_per_100kcal"].values
sugar   = df["free_sugar_g_per_100kcal"].values
sodium  = df["sodium_mg_per_100kcal"].values


In [216]:
# === Objective: minimize total food portions ===
# c = np.ones(len(df))  # minimize number of 100 kcal servings
# Adjusted objective function (minimize portions + sugar + sodium)
c = np.ones(len(df)) + 0.01 * sugar + 0.001 * sodium


In [217]:
# === Constraints: protein, carbs, fat (≥), fiber (≥), sugar (≤), sodium (≤) ===
A = [
    -protein,   # ≥ protein_target
    -carbs,     # ≥ carbs_target
    -fat,       # ≥ fat_target
    -fiber,     # ≥ 25g
    sugar,      # ≤ 25g
    sodium      # ≤ 2300mg
]
b = [
    -protein_target,
    -carbs_target,
    -fat_target,
    -25,    # min fiber
    25,     # max sugar
    2300    # max sodium
]

In [218]:
# === Bounds: (0, 1) for condiments, (0, 4) for regular foods ===
bounds = list(df["portion_bounds"])

In [219]:
# bounds

In [220]:
# === Run optimization ===
result = linprog(c=c, A_ub=A, b_ub=b, bounds=bounds, method="highs")

In [221]:
if result.success:
    df["Portions_100kcal"] = result.x.round(2)
    selected = df[df["Portions_100kcal"] > 0].copy()

    # === Compute total nutrients ===
    selected["Calories"]            = selected["Portions_100kcal"] * 100
    selected["Total Protein (g)"]   = selected["Portions_100kcal"] * selected["protein_g_per_100kcal"]
    selected["Total Carbs (g)"]     = selected["Portions_100kcal"] * selected["carbohydrates_g_per_100kcal"]
    selected["Total Fat (g)"]       = selected["Portions_100kcal"] * selected["fats_g_per_100kcal"]
    selected["Total Fiber (g)"]     = selected["Portions_100kcal"] * selected["fibre_g_per_100kcal"]
    selected["Total Sugar (g)"]     = selected["Portions_100kcal"] * selected["free_sugar_g_per_100kcal"]
    selected["Total Sodium (mg)"]   = selected["Portions_100kcal"] * selected["sodium_mg_per_100kcal"]

    # === Display result ===
    display(selected[[
        "dish_name", "Portions_100kcal", "Calories",
        "Total Protein (g)", "Total Carbs (g)", "Total Fat (g)",
        "Total Fiber (g)", "Total Sugar (g)", "Total Sodium (mg)"
    ]])
else:
    print("❌ Optimization failed:", result.message)


,dish_name,Portions_100kcal,Calories,Total Protein (g),Total Carbs (g),Total Fat (g),Total Fiber (g),Total Sugar (g),Total Sodium (mg)
551,Baked stuffed fish,4.00,400.0,52.935389,6.953099,17.809118,1.246310,1.147917,243.883240
600,Tomato aspic,1.87,187.0,25.505063,18.636887,0.965257,5.865793,11.917213,364.458805
849,Lemon chicken,0.48,48.0,5.710584,0.155486,2.730903,0.045232,0.045232,22.596384
956,Cracked wheat and green gram dal premix (Dalia...,4.00,400.0,17.550254,71.515352,5.643914,12.878286,0.695825,3.589574
957,Rice flakes and roasted channa,2.56,256.0,8.116137,55.238872,2.257469,6.787762,1.005879,157.800120
1013,Bengal 5 Spice Blend (Panch Phoran),4.00,400.0,25.204458,27.606198,30.587667,25.397702,1.932434,70.989337


In [222]:
# 🧮 Daily Nutrition Summary
total_cals   = selected["Calories"].sum()
total_prot   = selected["Total Protein (g)"].sum()
total_carbs  = selected["Total Carbs (g)"].sum()
total_fat    = selected["Total Fat (g)"].sum()
total_fiber  = selected["Total Fiber (g)"].sum()
total_sugar  = selected["Total Sugar (g)"].sum()
total_sodium = selected["Total Sodium (mg)"].sum()

print("\n🧮 Your Daily Nutrition Summary:")
print(f"Calories:      {round(total_cals, 1)} kcal")
print(f"Protein:       {round(total_prot, 1)} g")
print(f"Carbs:         {round(total_carbs, 1)} g")
print(f"Fat:           {round(total_fat, 1)} g")
print(f"Fiber:         {round(total_fiber, 1)} g   (Min: 25 g)")
print(f"Free Sugar:    {round(total_sugar, 1)} g   (Max: 25 g)")
print(f"Sodium:        {round(total_sodium, 1)} mg (Max: 2300 mg)")

# Optional: Alerts
if total_fiber < 25:
    print("⚠️ Low fiber intake — consider adding vegetables or legumes.")
if total_sugar > 25:
    print("⚠️ High sugar intake — reduce sweets or processed items.")
if total_sodium > 2300:
    print("⚠️ High sodium intake — may impact heart health.")



🧮 Your Daily Nutrition Summary:
Calories:      1691.0 kcal
Protein:       135.0 g
Carbs:         180.1 g
Fat:           60.0 g
Fiber:         52.2 g   (Min: 25 g)
Free Sugar:    16.7 g   (Max: 25 g)
Sodium:        863.3 mg (Max: 2300 mg)


In [223]:
# === Display result with additional info ===
display(selected[[  
    "dish_name", "veg_nonveg", "meal_time", "Portions_100kcal", "Calories", 
    "Total Protein (g)", "Total Carbs (g)", "Total Fat (g)", 
    "Total Fiber (g)", "Total Sugar (g)", "Total Sodium (mg)"
]])


,dish_name,veg_nonveg,meal_time,Portions_100kcal,Calories,Total Protein (g),Total Carbs (g),Total Fat (g),Total Fiber (g),Total Sugar (g),Total Sodium (mg)
551,Baked stuffed fish,Non-Veg,Lunch/Dinner,4.00,400.0,52.935389,6.953099,17.809118,1.246310,1.147917,243.883240
600,Tomato aspic,Veg,Any,1.87,187.0,25.505063,18.636887,0.965257,5.865793,11.917213,364.458805
849,Lemon chicken,Non-Veg,"Lunch, Dinner",0.48,48.0,5.710584,0.155486,2.730903,0.045232,0.045232,22.596384
956,Cracked wheat and green gram dal premix (Dalia...,Veg,Breakfast,4.00,400.0,17.550254,71.515352,5.643914,12.878286,0.695825,3.589574
957,Rice flakes and roasted channa,Veg,Breakfast,2.56,256.0,8.116137,55.238872,2.257469,6.787762,1.005879,157.800120
1013,Bengal 5 Spice Blend (Panch Phoran),Veg,Lunch/Dinner,4.00,400.0,25.204458,27.606198,30.587667,25.397702,1.932434,70.989337
